In [1]:
from modeling.couplingapi import Subproblem
from modeling.api import Var
import numpy as np

In [2]:
A = np.array([[0, 0.3970, 0.8152, 0.9230, 0.1108],
              [0.4252, 0, 0.6357, 0.7435, 0.1138],
              [0.0329, 0.8856, 0, 0.3657, 0.0019],
              [0.0878, 0.7248, 0.1978, 0, 0.0169],
              [0.8955, 0.4568, 0.8075, 0.9239, 0]])

In [3]:
import numpy as np

coefficients = {
    'Theta': {'Ao': np.array([1.]), 'Ai': np.array([0.3, -0.3, -0.3, -0.2]), 'Aii': np.array([0.4, -0.4, -0.4, 0])},
    'Fol': {'Ao': np.array([1.]), 'Ai': np.array([6.25]), 'Aii': np.array([0])},
    'sigma1': {'Ao': np.array([1.]), 'Ai': np.array([-0.75, 0.5, -0.75, 0.5, 0.5]), 'Aii': np.array([-2.5, 0, -2.5, 0, 0])},
    'sigma2': {'Ao': np.array([1.]), 'Ai': np.array([-0.50, 0.333, -0.50, 0.333, 0.333]), 'Aii': np.array([-1.1110, 0, -1.111, 0, 0])},
    'sigma3': {'Ao': np.array([1.]), 'Ai': np.array([-0.3750, 0.25, -0.375, 0.250, 0.25]), 'Aii': np.array([-0.6250, 0, -0.62500, 0, 0, 0])},
    'sigma4': {'Ao': np.array([1.]), 'Ai': np.array([-0.3, 0.2, -0.3, 0.2, 0.2]), 'Aii': np.array([-0.4, 0, -0.4, 0, 0])},
    'sigma5': {'Ao': np.array([1.]), 'Ai': np.array([-0.250, 0.1667, -0.25, 0.16670, 0.1667]), 'Aii': np.array([-0.27780, -0.277800, 0, 0])},
    'Fo2': {'Ao': np.array([1.]), 'Ai': np.array([0.2, 0.2]), 'Aii': np.array([0, 0])},
    'Fo3': {'Ao': np.array([1.]), 'Ai': np.array([0]), 'Aii': np.array([0.04])},
    'dp/dx': {'Ao': np.array([1.]), 'Ai': np.array([0.2]), 'Aii': np.array([0])},
    'Temp': {'Ao': np.array([1.]), 'Ai': np.array([0.3, -0.3, 0.3]), 'Aii': np.array([0.4, -0.4, 0.4])}
}


In [4]:
def compute_function(A, allcoefficients, arg):
    # Initialize Aii and A_i as zero vectors of the correct size and fill in the non-zero entries
    coefficients = allcoefficients[arg]
    A0 = coefficients['Ao']
    Ai = coefficients['Ai']
    Aii = coefficients['Aii']
    dim_A = Ai.shape[0]

    Aij = np.diag(Aii)+ A[:dim_A,:dim_A]
    print(A0, Ai)
    print(Aij)
    def evalpoly(*S):
        S = np.array(S)
        return A0 + Ai.dot(S) + 0.5 * S.dot(Aij).dot(S.T)
    return evalpoly

In [29]:
compute_function(A, coefficients, 'Theta')(np.array([1, 2, 3, 4]))

[1.] [ 0.3 -0.3 -0.3 -0.2]
[[ 0.4     0.397   0.8152  0.923 ]
 [ 0.4252 -0.4     0.6357  0.7435]
 [ 0.0329  0.8856 -0.4     0.3657]
 [ 0.0878  0.7248  0.1978  0.    ]]


array([14.53405])

In [ ]:
eqs = '''
t = tc * S_REF / sqrt(S_REF * AR)\n
b_half = sqrt(S_REF * AR / 2)\n
R = (1 + 2 * lambda) / (3 * (1 + lambda))\n
theta = pf(x, b_half, R, L)\n
fo1 = pf(x)\n
W_W = (0.0051 * (W_T * N_z)**0.557 * S_REF*0.649 * AR**0.5 * (t/c)**-0.4 * (1+lambda)**0.1 * (0.1875S_REF)**0.1 / cos(lambda)) * Fo_1\n
W_FW = (5 * S_REF / 18) * (2/3 * t) * 42.5\n
W_F = W_FW + W_FO\n
W_T = W_O + W_W + W_F + W_E\n
sigma1_to_sigma5 = pf(t/c, L_val, x_val, b_half, R)\n
'''

In [ ]:
aero = '''
CL = WT / (0.5 * rho * V**2 * S_REF) \n
Fol = pf(ESF, Cf) \n 
CDmin = (CD_min * (M < 1) * F_0 * (1 + 3.05 * (1 / (k * S_REF))**5)**3 * cos(Lambda)**2)
k = 1 / (pi * 0.8 * AR) \n
Fo_0 = pf(Theta) \n
CD = (CD_m0 + k * CL**2) * Fo * 2 \n
L = WT \n
D = CD * 0.5 * rho * V**2 * S_REF \n
dp_dx = pf(t / c) \n
'''

In [ ]:
prop = '''
Tbar = T * 16168.6\n
Temp = pf(M, h, T)\n
ESF = (D / 3) / Tbar\n
SFC = 1.1324 + 1.5344 * M - 3.2956e-05 * h - 1.6379e-04 * Tbar - 0.31623 * M**2 + 8.2138e-06 * M * h - 10.496e-05 * T * M - 8.574e-11 * h**2 + 3.8042e-09 * Tbar * h + 1.0600e-08 * Tbar**2\n
WE = 3 * W_BE * ESF**1.05\n
T_UA = 11484 + 10856 * M - 0.50802 * h + 3200.2 * M**2 - 0.29326 * M * h + 6.8572e-06 * h**2\n
'''

In [ ]:
range = '''
theta = lambda h: 1 - 6.875e-06 * h if h < 36089 else 0.7519
R = (M * (L - D) * 66 * sqrt(theta)) / (SFC * log(W_T / (W_T - W_F)))
'''

In [5]:
from sympy import log, sqrt, exp

In [31]:
P1 = Subproblem()
M = Var('M')
L = Var('L')
D = Var('D')
SFC = Var('SFC')
W_T = Var('W_T')
W_F = Var('W_F')
h = Var('h')
theta = P1.add_equation('theta', lambda h: 1 - 6.875e-06 * h if h < 36089 else 0.7519, (h,))
R = P1.add_equation('R', (M * (L - D) * 66 * sqrt(theta)) / (SFC * log(W_T / (W_T - W_F))))

In [32]:
from modeling.execution import edges_from_components

In [37]:
edges = edges_from_components(P1.components)

In [38]:
edges

({0: ('h',), 1: ('D', 'W_F', 'theta', 'L', 'W_T', 'SFC', 'M')},
 {0: ('theta',), 1: ('R',)},
 {})

In [16]:
from modeling.api import evalexpr, sympy_fx_inputs, Component, calculateval

In [8]:
V = M * 968.1
rho = (2.377e-03) * (.2971) * exp(-(M-36089) / 20806.7)
Sref = Var(r'S_{ref}', 1000, 'ft^2')

In [22]:
comp = Component.fromsympy(0.5*0.5*rho*V**2*Sref, Var('',unit='ft**2'), arg_mapping=True)

In [24]:
calculateval(comp.mapped_inputs, comp.function)

(array(3037378.28977385), {})

In [26]:
evalexpr(0.5*0.5*rho*V**2*Sref)

dummy

In [34]:
T = Var('T', 0.5) #Throttle setting
M = Var('M', 1.8)
h = Var('h', 60000)
D = Var('D', 2.82e5)
W_BE = Var(r'W_{BE}', 4360.0)

In [35]:
P1 = Subproblem()
Tbar = P1.add_equation(r'T_{bar}', T * 16168.6)
pf = compute_function(A, coefficients, 'Temp')
Temp = P1.add_equation('Temp', pf, (M, h, T))
ESF = P1.add_equation('ESF', (D / 3) / Tbar)
SFC = P1.add_equation('SFC', 1.1324 + 1.5344 * M - 3.2956e-05 * h - 1.6379e-04 * Tbar - 0.31623 * M**2 + 8.2138e-06 * M * h - 10.496e-05 * T * M - 8.574e-11 * h**2 + 3.8042e-09 * Tbar * h + 1.0600e-08 * Tbar**2)
WE = P1.add_equation('WE', 3 * W_BE * ESF**1.05)
TUA = P1.add_equation(r'T_{UA}', 11484 + 10856 * M - 0.50802 * h + 3200.2 * M**2 - 0.29326 * M * h + 6.8572e-06 * h**2)

[1.] [ 0.3 -0.3  0.3]
[[ 0.4     0.397   0.8152]
 [ 0.4252 -0.4     0.6357]
 [ 0.0329  0.8856  0.4   ]]


In [40]:
pf(M.varval, h.varval, T.varval)

array([-7.19950779e+08])

In [43]:
Ao = coefficients['Temp']['Ao']
Ai = coefficients['Temp']['Ai']
Aii = coefficients['Temp']['Aii']

In [44]:
S = np.array([M.varval, h.varval, T.varval])

In [51]:
0.5*S@(A[:3,:3]+np.diag(Aii))@S

-719932780.620355

## Poly Approx

In [70]:
import numpy as np

def precompute_poly_approx(S, flag, S_bound):
    size = len(S)
    a, b = 0.1, 0.1
    Ao, Ai, Aij = 0, np.zeros(size), np.zeros((size, size))

    for i in range(size):
        if flag[i] == 5:
            So, Sl, Su = 0, -S_bound[i], S_bound[i]
            Mtx_shifted = np.array([[1, S[i], S[i]**2], [1, So, So**2], [1, Su, Su**2]])
            F_bound = np.array([1 + (0.5 * a)**2, 1, 1 + (0.5 * b)**2])
        else:
            if flag[i] == 3:
                a, b = -a, -b
            elif flag[i] == 2:
                b = 2 * a
            elif flag[i] == 4:
                a, b = -a, 2 * a
            
            So, Sl, Su = 0, -S_bound[i], S_bound[i]
            Mtx_shifted = np.array([[1, S[i], Sl**2], [1, So, So**2], [1, Su, Su**2]])
            F_bound = np.array([1 - 0.5 * a, 1, 1 + 0.5 * b])
        
        A = np.linalg.solve(Mtx_shifted, F_bound)
        Ao, Ai[i], Aij[i, i] = A[0], A[1], A[2]

    R = np.array([[0.2736, 0.3970, 0.8152, 0.9230, 0.4252],
                  [0.4415, 0.6357, 0.7435, 0.0329, 0.8856],
                  [0.8390, 0.3657, 0.0878, 0.7248, 0.1978],
                  [0.0200, 0.8955, 0.4568, 0.8075, 0.9239],
                  [0.1108, 0.1138, 0.0019, 0.0169, 0.2525]])

    for i in range(size):
        for j in range(i+1, size):
            Aij[i, j] = Aij[i, i] * R[i, j]
            Aij[j, i] = Aij[i, j]

    return Ao, Ai, Aij

def calculate_FF(S_new, S, Ao, Ai, Aij):
    S_norm = np.minimum(1.25, np.maximum(0.75, S_new / S))
    S_shifted = S_norm - 1
    FF = Ao + np.dot(Ai, S_shifted) + 0
    return FF


In [71]:
Ao,Ai,Aij = precompute_poly_approx([1.6, 45000, 0.5], [2,4,2], [.25,.25,.25])

In [85]:
i0=[.25, 1, 1, .5, .05, 45000, 1.6, 5.5, 55, 1000];
x_initial=i0[1];
tc_initial=i0[4];
L_initial=sqrt(i0[7]*i0[9])/2;
R_initial=( 1 +2*i0[1])/(3*(1 +i0[1]));

In [ ]:
Ao,Ai,Aij = precompute_poly_approx([x_initial, L_initial, R_initial], [2,4,4,3], [.25,.25,.25,.25])

In [84]:
Ao,Ai,Aij

(1.0,
 array([-1.11111111e-01, -1.11111728e-06,  6.00000000e-01]),
 array([[ 2.04444444,  0.81164444,  1.66663111],
        [ 0.81164444,  1.60000444,  1.1896033 ],
        [ 1.66663111,  1.1896033 , -4.        ]]))

In [83]:
calculate_FF(np.array([2.5, 60000, 0.8]), 
             np.array([1.6, 45000, 0.5]), Ao, Ai, Aij)

1.1222219444429011